In [38]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [39]:
url_1974 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_1974_United_Kingdom_general_elections'
url_1979 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_1979_United_Kingdom_general_election'
url_1983 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_1983_United_Kingdom_general_election'
url_1987 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_1987_United_Kingdom_general_election'
url_1992 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_1992_United_Kingdom_general_election'
url_1997 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_1997_United_Kingdom_general_election'
url_2001 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2001_United_Kingdom_general_election'
url_2005 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2005_United_Kingdom_general_election'
url_2010 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2010_United_Kingdom_general_election'
url_2012 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2015_United_Kingdom_general_election_(2010%E2%80%932012)'
url_2015 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2015_United_Kingdom_general_election'
url_2017 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2017_United_Kingdom_general_election'
url_2019 = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_2019_United_Kingdom_general_election'

url = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_next_United_Kingdom_general_election'

In [40]:
general_elections = [
    url_1974,
    url_1979,
    url_1983,
    url_1987,
    url_1992,
    url_1997,
    url_2001,
    url_2005,
    url_2010,
    url_2012,
    url_2015,
    url_2017,
    url_2019,
    url
]


In [41]:
election_tables = []
page = requests.get(url_1974)
soup = BeautifulSoup(page.content, "html.parser")
election_tables.append(('1974', pd.read_html(str(soup.find(id='October_general_election').find_next('table')))[0]))
for election in general_elections:
    page = requests.get(election)
    soup = BeautifulSoup(page.content, "html.parser")
    toc = soup.find(id='toc')
    level2 = toc.find_all("li", {"class": "toclevel-2"})
    level2_links = [x.find('a')['href'] for x in level2]
    years = [x.replace('#', '') for x in level2_links if len(x) == 5]
    for year in years:
        if election == url_2015 and year in ['2010', '2011', '2012']:
            continue
        table = pd.read_html(str(soup.find(id=year).find_next('table')))[0]
        if len(table) < 20:
            break
        election_tables.append((year, table))

In [42]:
column_rename = {'All': 'alliance',
                 'Area': 'area',
                 'Brexit': 'brexit_party',
                 'Change UK': 'change_uk',
                 'Client': 'client',
                 'Con': 'conservative',
                 'Date(s)Conducted': 'dates',
                 'Date(s)conducted': 'dates',
                 'Datesconducted': 'dates',
                 'Green': 'green',
                 'Lab': 'labour',
                 'Lead': 'lead',
                 'Lib': 'liberal',
                 'Lib Dem': 'liberal_democrat',
                 'Other': 'others',
                 'Others': 'others',
                 'Plaid Cymru': 'plaid_cymru',
                 'Polling Organisation / Client': 'pollster_client',
                 'Polling organisation/client': 'pollster_client',
                 'Pollster': 'pollster',
                 'Pollster/Client': 'pollster_client',
                 'Pollster/client(s)': 'pollster_client',
                 'Reform': 'reform_uk',
                 'SDP': 'social_democratic_party',
                 'SLD': 'liberal_democrat',
                 'SNP': 'scottish_national_party',
                 'Sample size': 'sample_size',
                 'Samplesize': 'sample_size',
                 'Surveyend date': 'dates',
                 'UKIP': 'united_kingdom_independence_party',
                 'Reclaim Party': 'reclaim_party',
                 'Brexit Party': 'brexit_party'
                 }


In [43]:
poll_info_columns = [
    'dates',
    'year',
    'area',
    'pollster',
    'pollster_client',
    'client',
    'sample_size']
party_columns = [
    'alliance',
    'brexit_party',
    'change_uk',
    'conservative',
    'green',
    'labour',
    'liberal',
    'liberal_democrat',
    'plaid_cymru',
    'reform_uk',
    'scottish_national_party',
    'social_democratic_party',
    'united_kingdom_independence_party',
    'reclaim_party']
final_columns = [
    'others',
    'lead']
column_names = poll_info_columns + party_columns + final_columns


In [44]:
def set_others(a):
    if pd.isna(a):
        return {'others': np.nan}
    b = a.split(' on')
    i = 1
    results = {}
    if len(b) == 1:
        results['others'] = b[0].split('%')[0].strip() + '%'
        return results
    while len(b) > i:
        party = b[i - 1].split('%')[1].strip()
        if party in column_rename:
            party = column_rename[party]
        else:
            print(party, a)
        result = b[i].split('%')[0].strip()
        results[party] = result + '%'
        i += 1
    return results

In [49]:
all_polls = pd.DataFrame(columns=column_names)
for year, polling_results in election_tables:
    table = polling_results.copy()
    table.columns = [column_rename[a] for a, b in table.columns]
    table['year'] = year
    all_polls = all_polls.append(table, ignore_index=True)
all_polls = all_polls[~((all_polls['lead'] == all_polls['conservative']) & (all_polls['lead'] == all_polls['labour']))]

In [50]:
for column in column_names:
    all_polls[column] = all_polls[column].str.replace(r"\[.*\]", "")

C:\Users\vtxvt\AppData\Local\Temp/ipykernel_36116/2571999208.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  all_polls[column] = all_polls[column].str.replace(r"\[.*\]", "")


In [51]:
for party in party_columns:
    replacement_values = {'?': np.nan, '–': np.nan, '–': np.nan, '-': np.nan, '*': np.nan, 'TBA': np.nan}
    all_polls[party] = pd.to_numeric(
        all_polls[party]
            .str.strip('%')
            .str.strip('<')
            .str.strip('>')
            .replace(replacement_values)
    )


In [52]:
all_polls['lead'] = pd.to_numeric(all_polls['lead'].str.strip('%').replace('Tie', 0))

In [53]:
other_parties = pd.json_normalize(all_polls['others'].map(set_others))

In [54]:
for party in other_parties.columns:
    other_parties[party] = pd.to_numeric(
        other_parties[party]
            .str.strip('%')
            .str.strip('<')
            .str.strip('>')
            .replace(replacement_values)
    )

    if party not in all_polls.columns or party == 'others':
        all_polls[party] = other_parties[party]
    else:
        all_polls[party] = all_polls[party].to_frame(name='a').join(other_parties[party]).sum(axis=1, min_count=1)

In [55]:
all_polls['date_started'] = np.nan
all_polls['date_concluded'] = np.nan

In [56]:
def set_date(date, year):
    date = date.replace('–', '-').replace('Pre-', '').strip()
    date_parts = date.split(" ")
    for part in date_parts:
        if len(part) == 4 and (part.startswith('19') or part.startswith('20')):
            year = part
            date_parts.remove(part)
    date = ' '.join(date_parts)
    if ' ' not in date:
        start_date = end_date = f'{year}-{date}-1'
    else:
        if '-' not in date:
            day, month = date.split(' ')
            start_date = end_date = f'{year}-{month[0:3]}-{day}'
        else:
            start, end = date.split('-')
            start = start.strip()
            end = end.strip()
            day, month = end.split(' ')
            end_date = f'{year}-{month[0:3]}-{day}'
            if ' ' in start:
                day, month = start.split(' ')
            else:
                day = start
            start_date = f'{year}-{month[0:3]}-{day}'
    return start_date, end_date

In [57]:
for ix, row in all_polls.iterrows():
    date = row.dates
    row.date_started, row.date_concluded = set_date(row.dates, row.year)
    if not pd.isna(row.pollster_client):
        pc = row.pollster_client.split('/')
        row.pollster = pc[0]
        if len(pc) > 1:
            row.client = pc[1]
    all_polls.loc[ix] = row

In [58]:
all_polls['date_started'] = pd.to_datetime(all_polls['date_started'])
all_polls['date_concluded'] = pd.to_datetime(all_polls['date_concluded'])

In [59]:
all_polls = all_polls[['date_started', 'date_concluded', 'area', 'pollster', 'client',
                       'sample_size', 'alliance', 'brexit_party', 'change_uk', 'conservative',
                       'green', 'labour', 'liberal', 'liberal_democrat', 'plaid_cymru',
                       'reform_uk', 'scottish_national_party', 'social_democratic_party',
                       'united_kingdom_independence_party', 'reclaim_party', 'others', 'lead']].copy()

In [60]:
all_polls['pollster'].fillna('NA', inplace=True)

In [61]:
pollster_cleanup = {
    'Populus Online': 'Populus',
    'ComRes-': 'Savanta ComRes',
    'ComRes': 'Savanta ComRes',
    'SavantaComRes': 'Savanta ComRes',
    'Ipsos MOR': 'Ipsos MORI',
    'MORI': 'Ipsos MORI',
    'Ipsos-MORI': 'Ipsos MORI',
    'Ipsos Mori': 'Ipsos MORI',
    'Harris': 'Harris Interactive',
    'Harris Interative': 'Harris Interactive',
    'Angus Reid Public Opinion': 'Angus Reid Strategies',
    'TNS BMRB': 'TNS',
    'Focaldata (MRP)': 'FocalData (MRP)',
    'ICM': 'ICM Research',
    'Kantar': 'Kantar Public',
    'BMG': 'BMG Research',
    'Lord Ashcroft': 'Lord Ashcroft Polls',
    'TNS-BMRB': 'TNS',
}
pollsters_used = []
for ix, row in all_polls.iterrows():
    if '/' in row.pollster:
        row.pollster, row.client = row.pollster.split('/')
    if 'general election' in row.pollster.lower():
        row.pollster = 'General Election'
    if row.pollster in pollster_cleanup:
        row.pollster = pollster_cleanup[row.pollster]
    if not row.pollster in pollsters_used:
        pollsters_used.append(row.pollster)
    all_polls.loc[ix] = row

In [62]:
all_polls.pollster = all_polls.pollster.str.strip()

In [63]:
all_polls.sort_values(by='date_concluded', inplace=True)

In [64]:
latest_polls = pd.DataFrame()
for pollster in all_polls.pollster.unique():
    pollster_data = all_polls.query(f'pollster == "{pollster}"')
    latest_poll = pollster_data.iloc[-1]
    latest_polls = latest_polls.append(latest_poll)


In [65]:
def format_lead(row):
    if pd.isna(row.labour):
        return ''
    largest_share = row[party_columns].max()
    if len(row[row == largest_share]) > 1:
        return 'Tie'
    second = sorted([a for a in row[party_columns] if not pd.isna(a)], reverse=True)[1]
    return f'{row[row == largest_share].index[0]}+{largest_share - second:.0f}'


In [66]:
latest_polls['party_leads'] = latest_polls.apply(format_lead, axis=1)


In [67]:
all_polls['party_leads'] = all_polls.apply(format_lead, axis=1)

In [68]:
latest_polls[['date_concluded', 'pollster', 'conservative', 'labour', 'party_leads']].sort_values(by='date_concluded',
                                                                                                  ascending=False)

,date_concluded,pollster,conservative,labour,party_leads
4584,2021-12-21,Focaldata,34.0,41.0,labour+7
4585,2021-12-20,Redfield & Wilton Strategies,31.0,39.0,labour+8
4586,2021-12-20,YouGov,30.0,36.0,labour+6
4588,2021-12-16,Savanta ComRes,34.0,38.0,labour+4
4589,2021-12-15,Find Out Now,30.0,38.0,labour+8
4592,2021-12-13,Kantar Public,34.0,38.0,labour+4
4594,2021-12-11,Survation,32.0,39.0,labour+7
4598,2021-12-10,Ipsos MORI,34.0,39.0,labour+5
4597,2021-12-10,Opinium,32.0,41.0,labour+9
4605,2021-12-04,Deltapoll,37.0,38.0,labour+1


In [80]:
all_polls.columns

Index(['date_started', 'date_concluded', 'area', 'pollster', 'client',
       'sample_size', 'alliance', 'brexit_party', 'change_uk', 'conservative',
       'green', 'labour', 'liberal', 'liberal_democrat', 'plaid_cymru',
       'reform_uk', 'scottish_national_party', 'social_democratic_party',
       'united_kingdom_independence_party', 'reclaim_party', 'others', 'lead',
       'party_leads'],
      dtype='object')

In [85]:
all_polls[
    (all_polls.conservative < 31)
    & ((all_polls.brexit_party.fillna(0)+all_polls.united_kingdom_independence_party.fillna(0)) < 10)
][
    ['date_concluded', 'pollster', 'conservative', 'labour', 'party_leads']].sort_values(by='date_concluded',
                                                                                         ascending=False)

,date_concluded,pollster,conservative,labour,party_leads
4586,2021-12-20,YouGov,30.0,36.0,labour+6
4589,2021-12-15,Find Out Now,30.0,38.0,labour+8
3776,2013-02-11,Ipsos MORI,30.0,42.0,labour+12
3778,2013-02-10,ICM Research,29.0,41.0,labour+12
3784,2013-02-04,YouGov,30.0,45.0,labour+15
...,...,...,...,...,...
497,1981-04-27,Gallup,30.0,38.0,labour+8
498,1981-04-13,NOP,30.0,34.5,labour+2
503,1981-03-23,Gallup,28.0,38.0,labour+6
504,1981-03-16,NOP,30.0,34.0,labour+2


In [89]:
conlow['poll_year'] = conlow['date_concluded'].apply(lambda x: x.year)

In [94]:
conlow.groupby('poll_year').size().sort_index(ascending=False)

poll_year
2021     2
2013     4
2009     1
2006     1
2005     8
2004     6
2003     6
2001    22
2000    12
1999    33
1998    36
1997    65
1996    56
1995    43
1994    46
1993    27
1992     3
1990     9
1986    10
1985     7
1982     4
1981    21
1973     1
dtype: int64

In [70]:
all_polls[(all_polls.liberal_democrat > 20)][
    ['date_concluded', 'pollster', 'conservative', 'labour', 'liberal_democrat', 'party_leads']].sort_values(
    by='date_concluded',
    ascending=False)

,date_concluded,pollster,conservative,labour,liberal_democrat,party_leads
4196,2019-10-18,Survation,32.0,24.0,21.0,conservative+8
4208,2019-10-01,YouGov,34.0,21.0,23.0,conservative+11
4209,2019-09-27,YouGov,33.0,22.0,21.0,conservative+11
4211,2019-09-25,Survation,27.0,24.0,22.0,conservative+3
4212,2019-09-25,YouGov,33.0,22.0,22.0,conservative+11
...,...,...,...,...,...,...
874,1992-04-03,Gallup,37.5,37.5,21.0,Tie
873,1992-04-03,Ipsos MORI,37.0,39.0,21.0,labour+2
876,1992-04-03,ICM Research,36.2,38.7,20.4,labour+2
878,1992-04-01,Gallup,38.0,37.5,20.5,conservative+0


In [72]:
display_columns = ['date_concluded', 'pollster', 'conservative', 'labour', 'liberal_democrat', 'party_leads']

all_polls[
    (all_polls['pollster'].isin(['YouGov', 'General Election'])) &
    (all_polls.liberal_democrat > 10)
].sort_values(by='date_concluded',ascending=False)[display_columns]

,date_concluded,pollster,conservative,labour,liberal_democrat,party_leads
4586,2021-12-20,YouGov,30.0,36.0,12.0,labour+6
4102,2019-12-12,General Election,44.7,33.0,11.8,conservative+12
4101,2019-12-12,General Election,43.6,32.1,11.6,conservative+12
5020,2019-12-12,General Election,44.7,32.9,11.8,conservative+12
5019,2019-12-12,General Election,43.6,32.1,11.6,conservative+12
...,...,...,...,...,...,...
1752,2001-06-07,General Election,31.7,40.7,18.3,labour+9
1393,1997-05-01,General Election,30.7,43.2,16.8,labour+13
1984,1997-05-01,General Election,30.7,43.2,16.8,labour+13
1751,1992-04-09,General Election,41.9,34.4,17.8,conservative+8


In [76]:
140/(1790*(1-0.03-0.18-0.11))

0.11501807426881366

In [77]:
437/(1790*(1-0.03-0.18-0.11))

0.3590207032533684